In [5]:
print('HI!')

HI!


In [6]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyMuPDFLoader, PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

In [7]:
load_dotenv()

True

In [8]:
PINECONE_API_KEY = 'pcsk_7JkuSY_M2DQBFQcxA6KY3NzRS3M6x96yPnYFhQYqGdnnRZg9SySxu2A4ajuwzsZ1PUCq8G'
PINECONE_API_ENV = "us-east-1-aws"
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [9]:
# Extract data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls= PyPDFLoader
    )
    documents = loader.load()
    return documents

In [10]:
extract_data = load_pdf("data/")

In [11]:
len(extract_data)

637

In [12]:
# split text into chunks
def text_split(extracted_data):
   #set chunk size and lap
   text_splitter =  RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
   # start to split data base on text_splitter
   text_chunks = text_splitter.split_documents(extracted_data)

   return text_chunks

In [13]:
chunks = text_split(extract_data)
len(chunks)

5860

In [14]:
# download embedding
def download_OpenAIEmbedings():
    embeddings = OpenAIEmbeddings(
        model="text-embedding-3-small",
        dimensions=1536)
    return embeddings

In [17]:
embeddings = download_OpenAIEmbedings()